In [3]:
import numpy as np
from scipy.sparse import eye as speye
from scipy.sparse.csgraph import laplacian
from sklearn.manifold.locally_linear import (
    barycenter_kneighbors_graph, null_space, LocallyLinearEmbedding)
from sklearn.metrics.pairwise import pairwise_distances, rbf_kernel
from sklearn.neighbors import NearestNeighbors
from scipy.linalg import eigh, svd, qr, solve
from scipy.sparse import eye, csr_matrix
from scipy.sparse.linalg import eigsh
from keras.datasets import mnist, fashion_mnist
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from keras.layers import Input, Dense, concatenate, Lambda
from keras.models import Model
from keras import optimizers, objectives
from keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.decomposition import PCA
import glob

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.astype('float32') / 255.
train_images = train_images.reshape((len(x_train), np.prod(train_images.shape[1:])))

In [14]:
input_img = Input(shape=(784,))
# encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(input_img)

bottlenek = Dense(10, activation='relu', name="bottleneck")(encoded)

decoded = Dense(64, activation='relu')(bottlenek)
# decoded = Dense(128, activation='relu')(decoded)
output = Dense(784, activation='sigmoid', name="logits")(decoded)


autoencoder = Model(input_img, output)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [15]:
autoencoder.fit(x_train, x_train,epochs=30)

Epoch 1/30
60000/60000 [==============================] - 8s 128us/step - loss: 0.2499
Epoch 2/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.1816
Epoch 3/30
60000/60000 [==============================] - 8s 131us/step - loss: 0.1662
Epoch 4/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.1547
Epoch 5/30
60000/60000 [==============================] - 7s 124us/step - loss: 0.1488
Epoch 6/30
60000/60000 [==============================] - 8s 127us/step - loss: 0.1456
Epoch 7/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.1433
Epoch 8/30
60000/60000 [==============================] - 8s 125us/step - loss: 0.1415
Epoch 9/30
60000/60000 [==============================] - 8s 127us/step - loss: 0.1400
Epoch 10/30
60000/60000 [==============================] - 8s 132us/step - loss: 0.1387
Epoch 11/30
60000/60000 [==============================] - 8s 133us/step - loss: 0.1376
Epoch 12/30
60000/60000 [================

In [8]:
bottleneck_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("bottleneck").output)

In [10]:
b = bottleneck_model.predict(x_train)

In [12]:
pd.DataFrame(b).to_csv("resultat/bottle_dae.csv", index=False, header=False)

## Lsta

In [28]:
from sklearn.manifold import LocallyLinearEmbedding

In [29]:
embedding = LocallyLinearEmbedding(n_components=2, method="ltsa")

In [30]:
X_transformed = embedding.fit_transform(x_train)

MemoryError: 

## NMF

In [20]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='nndsvd', random_state=0)
W = model.fit_transform(x_train)
H = model.components_

In [23]:
pd.DataFrame(W).to_csv("resultat/nmf.csv", index=False, header=False)

### MDS

In [16]:
from sklearn.manifold import MDS

In [17]:
embedding = MDS(n_components=2)
X_transformed = embedding.fit_transform(x_train)

MemoryError: 